In [2]:
# !pip install gliner

  Using cached gliner-0.2.13-py3-none-any.whl.metadata (7.3 kB)
  Using cached transformers-4.46.0-py3-none-any.whl.metadata (44 kB)
  Using cached huggingface_hub-0.26.1-py3-none-any.whl.metadata (13 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.20.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
Using cached gliner-0.2.13-py3-none-any.whl (47 kB)
Using cached huggingface_hub-0.26.1-py3-none-any.whl (447 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 1.1 MB/s eta 0:00:09
   --- ------------------------------------ 0.8/10.0 MB 1.3 MB/s eta 0:00:07
   ---- ----------------------------------- 1.0/10.0 MB 1.5 MB/s eta 0:00:06
   ------ --------------------------------- 1.6/10.0 MB 1.5 MB/s eta 0:00:06
  

In [4]:
# !pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=c4c81cf98296762b7563e5f1df2ca650b2a5c671db61c67dd25eabcd908b38de
  Stored in directory: c:\users\zakroum hicham\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [1]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

d:\anaconda3\envs\computer_vision\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]
d:\anaconda3\envs\computer_vision\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
import wikipedia
from tqdm import  tqdm

In [3]:
page = wikipedia.page("Tom Hanks",auto_suggest=False)
# page.content[:1000]

In [4]:
# import nltk
# nltk.download('punkt')
# nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

def split_into_sentence_chunks(text):
    return sent_tokenize(text)

chunks = split_into_sentence_chunks(page.content)
print(len(chunks),len(chunks[0]))


287 76


In [5]:
labels = ["award","location","organization","person","movie"]

In [6]:
awards,locs,orgs,persons,movies=[],[],[],[],[]
for text in tqdm(chunks):
    entities = model.predict_entities(text,labels=labels,threshold=0.7)
            
    for entity in entities:
        t,l=entity["text"],entity["label"]
        if l == "award":
            awards.append(t.lower())
        elif l == "location":
            locs.append(t.lower())
        elif l == "organization":
            orgs.append(t.lower())
        elif l == "person":
            persons.append(t.lower())
        elif l == "movie":
            movies.append(t.lower())

100%|██████████| 287/287 [03:06<00:00,  1.54it/s]


In [7]:
len(set(persons)),len(set(orgs)),len(set(locs)),len(set(movies)),len(set(awards))

(147, 83, 46, 94, 34)

In [35]:
# !pip install litellm

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   - -----------------------------

In [20]:
sys_prompt = """
Your task is to extract relationships between entities from a given sentence. You should focus on the entities provided in the LABELS list below.

For each relationship you find, return a JSON object in the format:
{{"subject": "value", "relationship": "value", "object": "value"}}

**Example of the required format:**
[
    {{"subject": "Hicham", "relationship": "is the father of", "object": "Hamza"}},
    {{"subject": "Hamza", "relationship": "is the CEO of", "object": "Marjan"}},
    {{"subject": "Tesla", "relationship": "is located in", "object": "Texas"}}
]

**Guidelines:**
1. Only include a JSON object if **all three fields—subject, relationship, and object—are populated**. 
2. Base your extractions exclusively on the entities listed under LABELS. 
3. The quality of your output impacts model performance, so accuracy is critical.

**LABELS:**
{labels}
"""


In [17]:
print(sys_prompt)


Your task is to extract relationships between entities from a given sentence. You should focus on the entities provided in the LABELS list below.

For each relationship you find, return a JSON object in the format:
{"subject": "value", "relationship": "value", "object": "value"}

**Example of the required format:**
[
    {"subject": "Hicham", "relationship": "is the father of", "object": "Hamza"},
    {"subject": "Hamza", "relationship": "is the CEO of", "object": "Marjan"},
    {"subject": "Tesla", "relationship": "is located in", "object": "Texas"}
]

**Guidelines:**
1. Only include a JSON object if **all three fields—subject, relationship, and object—are populated**. 
2. Base your extractions exclusively on the entities listed under LABELS. 
3. The quality of your output impacts model performance, so accuracy is critical.

**LABELS:**
{labels}



In [18]:
# env variable
# from dotenv import load_dotenv
# import os
# load_dotenv()
# os.environ['GROQ_API_KEY']

In [23]:
import re
def extract_json(s:str) -> str:
    # Regular expression to match text within brackets
    matches = re.findall(r'\[.*?\]', s, re.DOTALL)

    # Get the content inside the first match (assuming there’s only one set of brackets)
    if matches:
        content_inside_brackets = matches[0]
        return content_inside_brackets


In [27]:
import time
import json
from litellm import completion
labels_str = "["+",".join([i for i in labels])+"]"
all_triples = []
for chunk in tqdm(chunks):
    try:
        user_prompt = "Context:{text}\n\nTriples"
        response = completion(model="groq/llama3-8b-8192",
                            messages=[{ "content": sys_prompt.format(labels=labels_str),"role": "system"},
                                      { "content": user_prompt.format(text=chunk),"role": "user"}],
                                      max_tokens=1000
                                      )
        res = response.choices[0].message.content
        res = extract_json(res)
        triples = json.loads(res)
        all_triples.append(triples)
        time.sleep(3)
    except Exception as e:
        print(f"error as = {e}")


  0%|          | 0/287 [00:00<?, ?it/s]

  2%|▏         | 7/287 [00:21<11:42,  2.51s/it]

error as = Expecting value: line 3 column 1 (char 144)


  3%|▎         | 10/287 [00:28<10:12,  2.21s/it]

error as = Expecting value: line 3 column 1 (char 88)


...

 93%|█████████▎| 267/287 [14:30<00:50,  2.51s/it]

error as = Expecting value: line 4 column 1 (char 164)


 99%|█████████▊| 283/287 [15:21<00:09,  2.48s/it]

error as = Expecting value: line 3 column 83 (char 167)


100%|██████████| 287/287 [15:35<00:00,  3.26s/it]


In [28]:
labels_str

'[award,location,organization,person,movie]'

In [29]:
all_triples

[[{'subject': 'Thomas Jeffrey Hanks',
   'relationship': 'is an',
   'object': 'actor and filmmaker'}],
 [{'subject': 'he',
   'relationship': 'is one of the most popular and recognizable film stars',
   'object': 'worldwide'},
  {'subject': 'he',
   'relationship': 'is regarded as',
   'object': 'an American cultural icon'}],
 [{'subject': 'Hanks',
   'relationship': 'is the fourth-highest-grossing American',
   'object': 'actor'},
  {'subject': 'Hanks',
   'relationship': 'have grossed',
   'object': '$4.9 billion nationally'},
  {'subject': 'Hanks',
   'relationship': 'have grossed',
   'object': '$9.96 billion worldwide'}],
 [{'subject': 'Hanks',
   'relationship': 'made his breakthrough with',
   'object': 'leading roles in a series of comedies'},
  {'subject': 'Hanks', 'relationship': 'starred in', 'object': 'Splash'},
  {'subject': 'Hanks',
   'relationship': 'starred in',
   'object': 'The Money Pit'},
  {'subject': 'Hanks', 'relationship': 'starred in', 'object': 'Big'},
  {'s

In [30]:
def assign_color(n:str) -> int:
    colors= {
        "person":"#009a93",
        "location":"#9fb40a",
        "award":"#dc9403",
        "movie":"#997733",
        "organization":"#eadcc5"
    }

    if n.lower() in persons:
        return colors["person"]
    elif n.lower() in locs:
        return colors["location"]
    elif n.lower() in awards :
        return colors["award"]
    elif n.lower() in movies:
        return colors["movie"]
    elif n.lower() in orgs:
        return colors["organization"]
    return "red"
    
def assign_size(n:str) -> int:
    sizes= {
            "person":50,
            "location":20,
            "award":10,
            "movie":30,
            "organization":40
        }

    if n.lower() in persons:
        return sizes["person"]
    elif n.lower() in locs:
        return sizes["location"]
    elif n.lower() in awards :
        return sizes["award"]
    elif n.lower() in movies:
        return sizes["movie"]
    elif n.lower() in orgs:
        return sizes["organization"]
    return 20
    


In [40]:
# !pip install pyvis

   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---

In [31]:
from pyvis.network import Network
import networkx as nx

In [51]:
all_data = persons+locs+orgs+movies+awards
len(all_data)

717

In [55]:
G = nx.Graph()

for items in all_triples:
    for item in items:
        # if item["object"]  in all_data:
            node1 = item["subject"]
            node2 = item["object"]
            G.add_node(node1,title=node1,color = assign_color(node1),size=assign_size(node1))
            G.add_node(node2,title=node2,color = assign_color(node2),size=assign_size(node2))
            G.add_edge(node1,node2,title=item["relationship"],weight=4)

In [58]:
nt = Network(height="750px",width="100%",notebook=True)
nt.from_nx(G)

nt.force_atlas_2based(central_gravity=0.015,gravity=-31)
# nt.show_buttons(filter_=['physics'])
nt.write_html("graph.html")
# html = nt.generate_html()
# with open("graph.html","w") as f:
#     f.write(html) 
# nt.show("graph.html")


In [45]:
# from IPython.display import IFrame
# IFrame("graph.html",width=1000,height=800)